In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
train_data_dir = '/Users/ppr/Desktop/Project/Intern-Year2/Datasets/GrayDatasetsResizeAugmented/train'
val_data_dir = '/Users/ppr/Desktop/Project/Intern-Year2/Datasets/GrayDatasetsResizeAugmented/val'
test_data_dir = '/Users/ppr/Desktop/Project/Intern-Year2/Datasets/GrayDatasetsResizeAugmented/test'

In [3]:
# Data generator for training set
train_datagen = ImageDataGenerator(
	rescale=1./255
)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(640, 640),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical')

# Data generator for validation set
val_datagen = ImageDataGenerator(
    rescale=1./255
)
val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(640, 640),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical')

# Data generator for test set
test_datagen = ImageDataGenerator(
    rescale=1./255
)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(640, 640),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical')

Found 781 images belonging to 11 classes.
Found 147 images belonging to 11 classes.
Found 133 images belonging to 11 classes.


In [4]:
submodel_1 = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(640, 640, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1280, activation='relu')
])

submodel_2 = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(1280,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(11, activation='softmax')
])

model = tf.keras.Sequential([
    submodel_1,
    submodel_2
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 1280)              199821760 
                                                                 
 sequential_1 (Sequential)   (None, 11)                369803    
                                                                 
Total params: 200191563 (763.67 MB)
Trainable params: 200191563 (763.67 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [6]:
epochs = 50
steps_per_epoch = train_generator.n // train_generator.batch_size
validation_steps = val_generator.n // val_generator.batch_size

In [7]:
model.fit(train_generator,
          epochs=epochs,
          steps_per_epoch=steps_per_epoch,
          validation_data=val_generator,
          validation_steps=validation_steps)

Epoch 1/50


2023-05-31 04:23:18.980949: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


24/24 [==============================] - ETA: 0s - loss: 2.4596 - accuracy: 0.1335 

2023-05-31 04:36:05.915412: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


24/24 [==============================] - 787s 31s/step - loss: 2.4596 - accuracy: 0.1335 - val_loss: 2.3424 - val_accuracy: 0.3047
Epoch 2/50
24/24 [==============================] - 692s 29s/step - loss: 2.2201 - accuracy: 0.2266 - val_loss: 2.0885 - val_accuracy: 0.2734
Epoch 3/50
24/24 [==============================] - 672s 28s/step - loss: 2.0431 - accuracy: 0.2377 - val_loss: 2.0205 - val_accuracy: 0.2734
Epoch 4/50
24/24 [==============================] - 669s 28s/step - loss: 1.8876 - accuracy: 0.2897 - val_loss: 2.1405 - val_accuracy: 0.1484
Epoch 5/50
24/24 [==============================] - 671s 28s/step - loss: 1.7164 - accuracy: 0.3391 - val_loss: 1.9708 - val_accuracy: 0.3281
Epoch 6/50
24/24 [==============================] - 561s 23s/step - loss: 1.5145 - accuracy: 0.4406 - val_loss: 2.2551 - val_accuracy: 0.3359
Epoch 7/50
24/24 [==============================] - 331s 14s/step - loss: 1.3017 - accuracy: 0.4726 - val_loss: 1.9893 - val_accuracy: 0.4219
Epoch 8/50
24/24 

In [8]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.n // test_generator.batch_size)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

2023-05-31 09:36:20.840562: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4/4 [==============================] - 11s 3s/step - loss: 2.4441 - accuracy: 0.5000
Test Loss: 2.444065809249878
Test Accuracy: 0.5


In [9]:
model.save("scratchModel.h5")

/Users/ppr/miniforge3/envs/sophomore/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model.save("scratchModel.keras")